In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.9/184.9 MB 10.7 MB/s eta 0:00:0000:0100:01


In [2]:
closed_questions = [
    "Who is the director of Movie?",
    "Who directed Movie?",
    "Tell me who directed Movie?",
    "Who is the filmmaker of Movie?",
    "Which director took charge of Movie?",
    "Who's the person behind the direction of Movie?",
    "Who helmed the direction of Movie?",

    "Who is the screenwriter of Movie?",
    "Who wrote the screenplay for Movie?",
    "Which writer was responsible for Movie?",
    "Who's credited as the screenwriter for Movie?",

    "What is the MPAA film rating of Movie?",
    "What's the censorship rating of Movie by MPAA?",

    "What is the genre of Movie?",
    "Which genre does Movie belong to?",
]


recommendation_questions = [
    "Recommend movies similar to Movie.",
    "What are some films similar to Movie?",
    "Can you suggest movies that are like Movie?",
    "If I enjoyed Movie, what other movies would you suggest?",
    "Which films would you recommend if I loved Movie?",

    "Given that I like Movie, Movie, Movie, can you recommend some movies?",
    "Based on my liking for Movie, Movie, Movie, suggest some films.",
    "I enjoy watching Movie, Movie, and Movie. What else would you recommend?",

    "Recommend movies like Movie, Movie, and Movie",
    "Can you suggest films similar to Movie, Movie, and Movie?",
    "Based on my preference for Movie, Movie, and Movie, what else should I watch?",

    "I like Movie, so recommend similar movies",
    "If I enjoyed Movie, what other films would you suggest?",
    "Movies similar to Movie are what I'm looking for. Recommendations?",
]


when_questions = [
    "When was Movie released?",
    "On which date was Movie released?",
    "Can you tell me when Movie was released?",
    "Which day did Movie hit the theaters?",

    "What is the release date of Movie?",
    "Tell me the release date of Movie.",
    "When exactly was Movie made available to the public?",

    "Which year was Movie released?",
    "In what year did Movie come out?",

    "When did Movie come out?",
    "On which day was Movie first shown?",
    "When was the premiere of Movie?",
]

multimedia_questions = [
    "Show me a picture of Actor.",
    "Can you display a photo of Actor?",
    "I'd like to see a photograph of Actor.",
    "Provide me with a picture of Actor.",

    "What does Actor look like?",
    "Can you describe Actor's appearance?",
    "Tell me about Actor's physical appearance.",

    "Let me know what Actor looks like.",
    "Give me an idea of Actor's looks.",
    "Provide a description or image of Actor.",
]

questions = closed_questions + recommendation_questions + \
    when_questions + multimedia_questions
labels = [0]*len(closed_questions) + [1]*len(recommendation_questions) + \
    [2]*len(when_questions) + [3]*len(multimedia_questions)
len(multimedia_questions)

10

In [ ]:
questions = closed_questions + recommendation_questions + \
    when_questions + multimedia_questions
labels = [0]*len(closed_questions) + [1]*len(recommendation_questions) + \
    [2]*len(when_questions) + [3]*len(multimedia_questions)
labels

In [5]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")


def get_embedding(question):
    tokens = tokenizer(question, return_tensors='pt',
                       truncation=True, max_length=512, padding='max_length')
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

/home/liamt/Uni/Master/HS23/ATAI/speakeasy-python-client-library/speakeasy-python-client-library/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch
# Convert questions and labels to PyTorch datasets
from torch.utils.data import Dataset, DataLoader


class MovieQuestionDataset(Dataset):
    def __init__(self, questions, labels):
        self.questions = questions
        self.labels = labels
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.questions[idx], truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


dataset = MovieQuestionDataset(questions, labels)

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


: 

# SPACY

In [14]:
closed_questions_dev = [
    "Who is the director of 'Inception'?",
    "Who directed 'Pulp Fiction'?",
    "Tell me who directed 'The Shape of Water'?",
    "Who is the filmmaker of 'Get Out'?",
    "Which director took charge of 'The Grand Budapest Hotel'?",
    "Who's the person behind the direction of 'Moonlight'?",
    "Who helmed the direction of 'Mad Max: Fury Road'?",
    "Who is the screenwriter of 'Eternal Sunshine of the Spotless Mind'?",
    "Who wrote the screenplay for 'Her'?",
    "Which writer was responsible for 'The Social Network'?",
    "Who's credited as the screenwriter for 'Juno'?",
    "What is the MPAA film rating of 'Joker'?",
    "What's the censorship rating of 'The Dark Knight' by MPAA?",
    "What is the genre of 'Blade Runner 2049'?",
    "Which genre does 'Parasite' belong to?",
]

recommendation_questions_dev = [
    "Recommend movies similar to 'The Matrix'.",
    "What are some films similar to 'Interstellar'?",
    "Can you suggest movies that are like 'Arrival'?",
    "If I enjoyed 'Blade Runner 2049', what other movies would you suggest?",
    "Which films would you recommend if I loved 'The Lord of the Rings'?",
    "Given that I like 'The Godfather', 'Goodfellas', 'Casino', can you recommend some movies?",
    "Based on my liking for 'Fight Club', 'Se7en', 'Gone Girl', suggest some films.",
    "I enjoy watching 'Amelie', 'La La Land', and 'The Grand Budapest Hotel'. What else would you recommend?",
    "Recommend movies like 'Alien', 'Prometheus', and 'The Martian'",
    "Can you suggest films similar to 'Saving Private Ryan', 'Platoon', and 'Full Metal Jacket'?",
    "Based on my preference for 'Forrest Gump', 'The Shawshank Redemption', and 'The Green Mile', what else should I watch?",
    "I like 'The Silence of the Lambs', so recommend similar movies",
    "If I enjoyed 'The Shining', what other films would you suggest?",
    "Movies similar to 'Psycho' are what I'm looking for. Recommendations?",
]

when_questions_dev = [
    "When was 'Titanic' released?",
    "On which date was 'The Lion King' released?",
    "Can you tell me when 'Schindler's List' was released?",
    "Which day did 'Jurassic Park' hit the theaters?",
    "What is the release date of 'Avatar'?",
    "Tell me the release date of 'The Avengers'.",
    "When exactly was 'Harry Potter and the Philosopher's Stone' made available to the public?",
    "Which year was 'Star Wars: Episode IV - A New Hope' released?",
    "In what year did 'Back to the Future' come out?",
    "When did 'Indiana Jones and the Raiders of the Lost Ark' come out?",
    "On which day was 'The Empire Strikes Back' first shown?",
    "When was the premiere of 'The Godfather'?",
]

multimedia_questions_dev = [
    "Show me a picture of Leonardo DiCaprio.",
    "Can you display a photo of Jennifer Lawrence?",
    "I'd like to see a photograph of Brad Pitt.",
    "Provide me with a picture of Meryl Streep.",
    "What does Denzel Washington look like?",
    "Can you describe Angelina Jolie's appearance?",
    "Tell me about Tom Hanks's physical appearance.",
    "Let me know what Charlize Theron looks like.",
    "Give me an idea of Johnny Depp's looks.",
    "Provide a description or image of Natalie Portman.",
]

In [15]:

# Combine all questions into one list
questions = closed_questions + recommendation_questions + \
    when_questions + multimedia_questions
questions_dev = closed_questions_dev + recommendation_questions_dev + \
    when_questions_dev + multimedia_questions_dev

# Create labels (0 for closed_questions, 1 for recommendation_questions, etc.)
labels = ['CLOSED']*len(closed_questions) + \
         ['RECOMMENDATION']*len(recommendation_questions) + \
         ['WHEN']*len(when_questions) + \
         ['MULTIMEDIA']*len(multimedia_questions)

# Define category names
category_names = ['CLOSED', 'RECOMMENDATION', 'WHEN', 'MULTIMEDIA']

# Create the spaCy training data format
TRAIN_DATA = [
    (question, {
     "cats": {label: 1.0 for label in category_names if label == cat_label}})
    for question, cat_label in zip(questions, labels)
]


labels = ['CLOSED']*len(closed_questions_dev) + \
         ['RECOMMENDATION']*len(recommendation_questions_dev) + \
         ['WHEN']*len(when_questions_dev) + \
         ['MULTIMEDIA']*len(multimedia_questions_dev)

# Define category names
category_names = ['CLOSED', 'RECOMMENDATION', 'WHEN', 'MULTIMEDIA']

# Create the spaCy training data format
DEV_DATA = [
    (question, {
     "cats": {label: 1.0 for label in category_names if label == cat_label}})
    for question, cat_label in zip(questions_dev, labels)
]

# Just to check a few of the training data
TRAIN_DATA
DEV_DATA

[("Who is the director of 'Inception'?", {'cats': {'CLOSED': 1.0}}),
 ("Who directed 'Pulp Fiction'?", {'cats': {'CLOSED': 1.0}}),
 ("Tell me who directed 'The Shape of Water'?", {'cats': {'CLOSED': 1.0}}),
 ("Who is the filmmaker of 'Get Out'?", {'cats': {'CLOSED': 1.0}}),
 ("Which director took charge of 'The Grand Budapest Hotel'?",
  {'cats': {'CLOSED': 1.0}}),
 ("Who's the person behind the direction of 'Moonlight'?",
  {'cats': {'CLOSED': 1.0}}),
 ("Who helmed the direction of 'Mad Max: Fury Road'?",
  {'cats': {'CLOSED': 1.0}}),
 ("Who is the screenwriter of 'Eternal Sunshine of the Spotless Mind'?",
  {'cats': {'CLOSED': 1.0}}),
 ("Who wrote the screenplay for 'Her'?", {'cats': {'CLOSED': 1.0}}),
 ("Which writer was responsible for 'The Social Network'?",
  {'cats': {'CLOSED': 1.0}}),
 ("Who's credited as the screenwriter for 'Juno'?", {'cats': {'CLOSED': 1.0}}),
 ("What is the MPAA film rating of 'Joker'?", {'cats': {'CLOSED': 1.0}}),
 ("What's the censorship rating of 'The Da

In [15]:
# Convert the questions and labels into the simpler tuple list format
SIMPLE_TRAIN_DATA = [(text, cat_label)
                     for text, cat_label in zip(questions, labels)]

# Check the first few entries to ensure they are correct
SIMPLE_TRAIN_DATA[:5]

[('Who is the director of Movie?', 'CLOSED'),
 ('Who directed Movie?', 'CLOSED'),
 ('Tell me who directed Movie?', 'CLOSED'),
 ('Who is the filmmaker of Movie?', 'CLOSED'),
 ('Which director took charge of Movie?', 'CLOSED')]

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

# We will use the previously formatted TRAIN_DATA to create the .spacy file
# The nlp object is created as a blank English model
nlp = spacy.load("en_core_web_sm")

# The DocBin is used to store the documents
db = DocBin()

for text, annotations in DEV_DATA:
    # Create a Doc object from the text and add the text categories
    doc = nlp(text)
    doc.cats = annotations["cats"]
    db.add(doc)

# Save the DocBin to a .spacy file
spacy_file_path = "./dev.spacy"
db.to_disk(spacy_file_path)

spacy_file_path  # Returning the file path to the user

In [59]:
nlp1 = spacy.load("output/model-last/")  # load the best model
# input sample text
doc = nlp1("What does Angelina Jolie look like? ")

print(doc.cats)  # print the categories

{'CLOSED': 0.12524087727069855, 'RECOMMENDATION': 0.06872432678937912, 'WHEN': 0.05711320787668228, 'MULTIMEDIA': 0.7489216327667236}
